# Introducción

En este cuaderno, se aborda el problema de la clasificación supervisada utilizando el algoritmo Naive Bayes. El enfoque se centra en trabajar con una base de datos que contiene variables continuas y una variable objetivo denominada "clase", que representa las etiquetas de clasificación.

El desafío principal al aplicar Naive Bayes a datos continuos se encuentra en la suposición del algoritmo de que las características son discretas. Para efrentar este reto, nuestro primer paso será discretizar las variables continuas utilizando el algoritmo CAIM (Class-Attribute Interdependence Maximization). Este método de discretización busca maximizar la dependencia entre las características y la clase objetivo, lo que resulta en una representación en forma de pertenencia a un intervalo para que sea legible en el clasificador Naive Bayes.

Una vez discretizados los datos, se implementa el clasificador Naive Bayes. Para evaluar la eficacia del modelo, se utiliza la técnica de validación cruzada K-Fold. Esto permite estimar la precisión del clasificador al dividir el conjunto de datos en 'K' subconjuntos, entrenando el modelo en 'K-1' de ellos y validando el rendimiento en el resto. Repitiendo este proceso 'K' veces y promediando los resultados, obtenemos una evaluación más robusta.


Primero hay que importar algunas librerias de python

In [1]:
import numpy as np
from collections import defaultdict
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import shuffle

## CAIM (Class-Attribute Interdependence Maximization)

El algoritmo CAIM (Class-Attribute Interdependence Maximization) es una técnica de discretización utilizada para convertir variables continuas en categóricas. Este algoritmo busca maximizar la dependencia entre los atributos y la clase objetivo, lo que resulta en una mejor separación de las clases que conlleva a una mejora del rendimiento del clasificador.

En el artículo CAIM Discretization Algorithm de Lukasz A. and Kurgan Krzysztof J. Cios se muestra un pseudocódigo del proceso, en matlab ya se encuentra una implementación dividida en 3 archivos con funciones complementarias. Tomando como referencia el artículo y la implementación en matlab (https://la.mathworks.com/matlabcentral/fileexchange/24344-caim-discretization-algorithm) se muestra una implementación en python:


## Funcionamiento del Algoritmo CAIM:
* Inicialización: El algoritmo comienza con la creación de un esquema de división inicial para cada variable continua, que sólo contiene los valores mínimos y máximos de la característica.

* Búsqueda de Puntos de División: Para cada variable, el algoritmo busca puntos de división potenciales entre los valores únicos de la característica, excluyendo el mínimo y el máximo.

* Evaluación de Esquemas de División: Para cada punto de división potencial, se crea un nuevo esquema de división y se evalúa su CAIM. El CAIM es una medida que cuantifica qué tan buena es la división basada en la proporción de la clase mayoritaria en cada intervalo del esquema de división, puede interpretarse como la probabilidad de cada valor en aportar a la división.

* Selección del Mejor Esquema: El algoritmo selecciona el esquema de división que maximiza el CAIM. Si el CAIM mejora lo suficiente entonces se actualiza el esquema de división; de lo contrario se detiene la búsqueda para esa característica.

* Discretización: Una vez que se han determinado los esquemas de división para todas las variables, los datos se discretizan reemplazando los valores continuos por etiquetas categóricas que representan los intervalos a los que pertenecen definidos por los esquemas de división.

## Implementación en el Código:

La clase CAIMD implementa el algoritmo CAIM. Se puede inicializar con una lista de características categóricas o dejar que el algoritmo determine automáticamente cuáles deben ser tratadas como categóricas. Para este caso ya consideramos que la última columna representa las clases

* El método fit se encarga de encontrar los esquemas de división óptimos para cada variable continua. Utiliza métodos auxiliares como get_caim para calcular el CAIM de un esquema de división e index_from_scheme para convertir un esquema de división en índices de división.
    
* El método transform se utiliza para discretizar un conjunto de datos basado en los esquemas de división encontrados durante el ajuste. Reemplaza los valores continuos en cada variable por etiquetas categóricas que representan los intervalos del esquema de división.

* El método fit_transform es una combinación de fit y transform para ajustar el modelo y luego transformar los datos en una sola operación.

    Se definen dos excepciones personalizadas, CategoricalParamException y NotEnoughPoints, para manejar situaciones específicas como parámetros incorrectos o características con valores insuficientes para la discretización.

Esta implementación permite discretizar características continuas de manera efectiva, preparándolas para su uso con clasificadores que requieren características categóricas, como Naive Bayes.

In [2]:
class CAIMD(BaseEstimator, TransformerMixin):

    def __init__(self, categorical_features='auto'):
        if isinstance(categorical_features, str):
            self._features = categorical_features
            self.categorical = None
        elif (isinstance(categorical_features, list)) or (isinstance(categorical_features, np.ndarray)):
            self._features = None
            self.categorical = categorical_features
        else:
            raise CategoricalParamException(
                "valor incorrecto de'categorical_features'. Ingrese 'auto', una matriz de índices o etiquetas.")

    def fit(self, X, y):
        ## Almacena los esquemas de división por características
        self.split_scheme = dict()
        print(self.split_scheme)
        if isinstance(X, pd.DataFrame ):
            if isinstance(self._features, list):
                self.categorical = [X.columns.get_loc(label) for label in self._features]
            X = X.values
            y = y.values
        if self._features == 'auto':
            self.categorical = self.check_categorical(X, y)
        categorical = self.categorical
        print('Categorical', categorical)

        min_splits = np.unique(y).shape[0]

        for j in range(X.shape[1]):
            if j in categorical:
                continue
            xj = X[:, j]
            xj = xj[np.invert(np.isnan(xj))]
            new_index = xj.argsort()
            xj = xj[new_index]
            yj = y[new_index]
            allsplits = np.unique(xj)[1:-1].tolist()  # potential split points
            global_caim = -1
            mainscheme = [xj[0], xj[-1]]
            best_caim = 0
            k = 1
            while (k <= min_splits) or ((global_caim < best_caim) and (allsplits)):
                split_points = np.random.permutation(allsplits).tolist()
                best_scheme = None
                best_point = None
                best_caim = 0
                k = k + 1
                #print(f"k ahora vale {k}")
                while split_points:
                    scheme = mainscheme[:]
                    sp = split_points.pop()
                    scheme.append(sp)
                    scheme.sort()
                    c = self.get_caim(scheme, xj, yj)
                    #print(f"probando con el scheme {scheme} con valor caim {c}\n")
                    if c > best_caim:
                        #print(f"el nuevo caim es mejor y ahora best_scheme es {scheme}")
                        best_caim = c
                        best_scheme = scheme
                        best_point = sp
                    
                if (k <= min_splits) or (best_caim > global_caim):
                    print("algo pasó")
                    mainscheme = best_scheme
                    global_caim = best_caim
                    try:
                        allsplits.remove(best_point)
                    except ValueError:
                        raise NotEnoughPoints('La variable' + str(j) + ' no cuenta con valores suficientes para discretizar')

            self.split_scheme[j] = mainscheme
            print('#', j, ' GLOBAL CAIM ', global_caim)
            print(self.split_scheme)
        return self

    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            self.indx = X.index
            self.columns = X.columns
            X = X.values
        X_di = X.copy()
        categorical = self.categorical

        scheme = self.split_scheme
        for j in range(X.shape[1]):
            if j in categorical:
                continue
            sh = scheme[j]
            sh[-1] = sh[-1] + 1
            xj = X[:, j]
            # xi = xi[np.invert(np.isnan(xi))]
            for i in range(len(sh) - 1):
                ind = np.where((xj >= sh[i]) & (xj < sh[i + 1]))[0]
                X_di[ind, j] = i
        if hasattr(self, 'indx'):
            return pd.DataFrame(X_di, index=self.indx, columns=self.columns)
        return X_di

    def fit_transform(self, X, y):
        self.fit(X, y)
        return self.transform(X)

    def get_caim(self, scheme, xi, y):
        sp = self.index_from_scheme(scheme[1:-1], xi)
        sp.insert(0, 0)
        sp.append(xi.shape[0])
        n = len(sp) - 1
        isum = 0
        for j in range(n):
            init = sp[j]
            fin = sp[j + 1]
            Mr = xi[init:fin].shape[0]
            val, counts = np.unique(y[init:fin], return_counts=True)
            maxr = counts.max()
            isum = isum + (maxr / Mr) * maxr
        return isum / n

    def index_from_scheme(self, scheme, x_sorted):
        split_points = []
        for p in scheme:
            split_points.append(np.where(x_sorted > p)[0][0])
        return split_points

    def check_categorical(self, X, y):
        categorical = []
        ny2 = 2 * np.unique(y).shape[0]
        for j in range(X.shape[1]):
            xj = X[:, j]
            xj = xj[np.invert(np.isnan(xj))]
            if np.unique(xj).shape[0] < ny2:
                categorical.append(j)
        return categorical


class CategoricalParamException(Exception):
    # Raise if wrong type of parameter
    pass


class NotEnoughPoints(Exception):
    # Raise if a feature must be categorical, not continuous
    pass

# Descripción del Clasificador Naive Bayes
El clasificador Naive Bayes es un algoritmo de aprendizaje automático supervisado basado en el teorema de Bayes. Ha mostrado ser eficiente para la clasificación de datos con características categóricas con la ventaja de ser simple y eficiente: Asume independencia condicional entre las características, lo que significa que el efecto de una característica en una clase es independiente del efecto de otras características evitando el cálculo de probabilidades combinando todas las variables.

## Funcionamiento del Clasificador Naive Bayes:

* Entrenamiento (Método fit): Durante la fase de entrenamiento, el clasificador calcula la probabilidad a priori de cada clase (la probabilidad de que un punto de datos pertenezca a una clase dada antes de observar las características) y las probabilidades de verosimilitud de las características dadas las clases (la probabilidad de observar una característica dada una clase).
$$P(y=c)=\frac{ registros \ con \ clase c}{total\ de\ datos}$$
$$P(x_i=v|y=c)= \frac{Elementos \ de \ la \ clase\ c \ con \ valor \ x_i =v}{Datos\ con\ clase\ c} $$

* Cálculo de Probabilidades de Verosimilitud (Método calculate_likelihood): Para cada característica, se calcula la probabilidad de verosimilitud como la frecuencia relativa de cada valor de la característica dentro de cada clase.

* Predicción (Método predict): Para un nuevo punto de datos, el clasificador calcula la probabilidad posterior de cada clase, que es proporcional al producto de la probabilidad a priori de la clase y las probabilidades de verosimilitud de las características observadas. La clase con la mayor probabilidad posterior se selecciona como la predicción.

$$P(y=c|x)\simeq P(y=c)\times \prod_{i=1}^n P(x_i|y=c) $$

## Validación Cruzada K-Fold
Para el split en training set y test set se utiliza validación cruzada k-fold: éste es una técnica utilizada para evaluar la eficacia de un modelo de aprendizaje automático y asegurar que es generalizable a nuevos registros. Consiste en dividir el conjunto de datos en 'K' subconjuntos (o "folds") de aproximadamente igual tamaño. El proceso se realiza de la siguiente manera:

* División de Datos: El conjunto de datos se divide aleatoriamente en 'K' subconjuntos o folds.

* Entrenamiento y Validación: En cada iteración, se utiliza un fold diferente como conjunto de validación, mientras que los 'K-1' folds restantes se utilizan como conjunto de entrenamiento. El modelo se entrena en el conjunto de entrenamiento y se valida en el conjunto de validación.

* Repetición: Este proceso se repite 'K' veces, de modo que cada fold se utiliza exactamente una vez como conjunto de validación.

Promedio de Resultados: Los resultados de las 'K' iteraciones se promedian para obtener una estimación más precisa del rendimiento del modelo.



## Implementación en el Código:

La clase NaiveBayes implementa el clasificador Naive Bayes para características categóricas.

* El método fit calcula y almacena las probabilidades a priori y de verosimilitud para cada clase y característica en el diccionario parameters. Las probabilidades a priori se calculan como la proporción de puntos de datos en cada clase, y las probabilidades de verosimilitud se calculan utilizando el método calculate_likelihood.

* El método calculate_likelihood calcula la probabilidad de verosimilitud de una característica como la frecuencia relativa de cada valor de la característica dentro de la clase.

* El método predict utiliza las probabilidades a priori y de verosimilitud almacenadas para calcular la probabilidad posterior de cada clase para un nuevo punto de datos. La clase con la mayor probabilidad posterior se selecciona como la predicción.

In [3]:
class NaiveBayes:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.parameters = {}
        for cls in self.classes:
            X_cls = X[y == cls]
            self.parameters[cls] = {
                'apriori': len(X_cls) / len(X),
                'verosim': {i: self.calculate_likelihood(X_cls[:, i]) for i in range(X.shape[1])}
            }

    def calculate_likelihood(self, feature):
        values, counts = np.unique(feature, return_counts=True)
        return dict(zip(values, counts / len(feature)))

    def predict(self, X):
        y_pred = []
        for x in X:
            posteriors = []
            for cls in self.classes:
                prior = self.parameters[cls]['apriori']
                likelihood = np.prod([self.parameters[cls]['verosim'][i].get(val, 1e-6) for i, val in enumerate(x)])
                posteriors.append(prior * likelihood)
            y_pred.append(self.classes[np.argmax(posteriors)])
        print(y_pred)
        return np.array(y_pred)

## El algoritmo en acción

Ahora, en la carpeta se encuentran 10 bases de datos a las cuales se puede acceder:
* bupa.csv 
    * Este conjunto de datos contiene información sobre pacientes que han sido sometidos a pruebas para detectar trastornos hepáticos. Tiene 6 variables continuas, que incluyen varias pruebas de función hepática como las concentraciones de enzimas en la sangre, y una variable de clase binaria que indica la presencia o ausencia de trastornos hepáticos.
* diabetes.csv 
    * Este conjunto de datos contiene información médica y de diagnóstico sobre pacientes femeninas de ascendencia indígena Pima. Tiene 8 variables continuas, como el número de embarazos, la concentración de glucosa en plasma, la presión arterial diastólica, y una variable de clase que indica si la paciente tiene diabetes.
* sonar.csv 
    * Este conjunto de datos contiene patrones de señales obtenidos al rebotar señales de sonar en diferentes superficies. Tiene 60 variables continuas que representan la energía en diferentes bandas de frecuencia, y una variable de clase que indica si la señal corresponde a una mina o a una roca.
* banknote.csv 
    * Este conjunto de datos contiene características extraídas de imágenes de billetes, como la variabilidad de la imagen, la entropía, etc. Tiene 4 variables continuas y una variable de clase que indica si el billete es auténtico o falso.
* heart.csv 
    * Este conjunto de datos contiene información sobre pacientes con enfermedades del corazón. Tiene 13 variables continuas, como la edad, el colesterol sérico, la presión arterial en reposo, y una variable de clase que indica la presencia de enfermedad cardíaca. (para este caso eliminé las variables discretas porque noté que el algoritmo las vuelve a modificar)

* iris.csv
    * El conjunto de datos Iris tiene cuatro variables continuas (longitud y ancho del sépalo, longitud y ancho del pétalo) y una variable de clase con tres categorías (especies de iris).
    
* musk_clean.csv
    * Este conjunto de datos describe un conjunto de 92 moléculas, con 162 variables, de las cuales 47 son consideradas almizcles por expertos humanos y las 45 restantes son consideradas no almizcles.  El objetivo es aprender a predecir si nuevas moléculas serán o no almizcles.
* rice.csv
    * Entre los arroces certificados cultivados en TURQUÍA, se han seleccionado para el estudio la especie Osmancik y la especie Cammeo, cultivada desde 2014.  Al observar las características generales de la especie Osmancik, éstas tienen un aspecto ancho, largo, vidrioso y opaco.  En cuanto a las características generales de la especie Cammeo, son anchas y largas, vidriosas y sin brillo.  Se tomó un total de 3.810 imágenes de granos de arroz de las dos especies, se procesaron y se hicieron inferencias de características. Se obtuvieron 7 rasgos morfológicos para cada grano de arroz
* sonar.csv
    * Este conjunto de datos contiene patrones de señales obtenidos al rebotar señales de sonar en diferentes superficies. Tiene 60 variables continuas que representan la energía en diferentes bandas de frecuencia, y una variable de clase que indica si la señal corresponde a una mina o a una roca.
* vehicle.csv
    * este conjunto de datos se utiliza para la clasificación de vehículos en cuatro categorías (autobús, furgoneta, automóvil, camión) basada en las características extraídas de las siluetas de los vehículos. Tiene 18 variables continuas, como el área de la silueta, la longitud máxima de la silueta, el ancho máximo de la silueta, la relación entre el área de la silueta y el cuadrado de la longitud de la silueta, y más. La variable de clase indica el tipo de vehículo. Para esta base decidí quitar algunas columnas que, aunque eran continuas, eran reescalamientos de otras variables.

* wine.csv
    * Este conjunto de datos contiene la información química de diferentes vinos. Tiene 13 variables continuas relacionadas con el contenido químico del vino y una variable de clase que indica el tipo de vino.



Sólo hay que cambiar el nombre de la base de datos y ejecutar todas las celdas de abajo para obtener el resultado

In [1]:
csv_file = "heart.csv" #cambiar el nombre a alguna otra base de datos

Primero se hace la discretización usando CAIM. Se especifica las variables y la clase target, se asume que la clase es la última columna.

Se hace un shuffle para aumentar la probabilidad de que el training set tenga al menos un elemento de cada clase

In [2]:
data = pd.read_csv(csv_file) # leer la base de datos

columnas = list(data.columns) # obtener la lista de las variables
variables = columnas[:-1] # todas las variables menos la última que es la clase
clase = columnas[-1] # la útlima columna es la clase

print("las variables son:")
for variable in variables:
    print(f"{variable}\n")
print("las clases son")
print(f"{data[clase].unique()}\n")

X = data[variables] # variables
y = data[clase] # target
               ###########################################
X= np.array(X) # la clase NaiveBayes recibe arrays       #
y= np.array(y) # por eso hay que cambiar el tipo de dato #
               ###########################################
    
X, y = shuffle(X, y, random_state=42) #se hace un shuffle para asegurar que training set tenga todas las clases

caim = CAIMD()  # se llama la clase caim
x_disc = caim.fit_transform(X, y)  # se discretiza

NameError: name 'pd' is not defined

Ahora vemos la base de datos original y en seguida la base discretizada

In [6]:
print(np.min(X[:,3]))

71.0


In [7]:
data

,age,trestbps,chol,thalach,oldpeak,target
0,63,145,233,150,2.3,1
1,37,130,250,187,3.5,1
2,41,130,204,172,1.4,1
3,56,120,236,178,0.8,1
4,57,120,354,163,0.6,1
...,...,...,...,...,...,...
298,57,140,241,123,0.2,0
299,45,110,264,132,1.2,0
300,68,144,193,141,3.4,0
301,57,130,131,115,1.2,0


In [8]:
df=pd.DataFrame(x_disc, columns=variables)

In [14]:
print(x_disc)

NameError: name 'X_disc' is not defined

Ahora se inicia un clasificador NaiveBayes y un kFolds con un split. Además se muestra la matriz de confusión:

In [9]:
clf = NaiveBayes()  # se crea el objeto con el que se va a entrenar
kf = KFold(n_splits=3) # se describe cómo se hara el split en kfolds

accuracies = []  #aquí guardamos la eficacia para hacer promedio
confusion_matrices = [] #aquí guardamos la matriz de confución de cada iteración

# Validación cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = x_disc[train_index], x_disc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    accuracies.append(accuracy_score(y_test, y_pred))
    cm = confusion_matrix(y_test, y_pred)
    confusion_matrices.append(cm)
    print(f"Matriz de confusión:\n{cm}\n")


[0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0]
Matriz de confusión:
[[31 11]
 [16 43]]

[1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1]
Matriz de confusión:
[[34 16]
 [17 34]]

[0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1]
Matriz

Ahora vemos el promedio de de precisión de cada fold y su desviación estandar

In [10]:
average_precision = np.mean(accuracies)
std_precision = np.std(accuracies)

print("Precisión promedio:", average_precision)
print("Desviación estándar de la presición:", std_precision)

Precisión promedio: 0.712871287128713
Desviación estándar de la presición: 0.028004228957883043


In [11]:
clf.parameters

{0: {'apriori': 0.45544554455445546,
  'verosim': {0: {0.0: 0.2608695652173913, 1.0: 0.7391304347826086},
   1: {0.0: 0.717391304347826, 1.0: 0.2826086956521739},
   2: {0.0: 0.5978260869565217, 1.0: 0.40217391304347827},
   3: {0.0: 0.6413043478260869, 1.0: 0.358695652173913},
   4: {0.0: 0.2826086956521739, 1.0: 0.717391304347826}}},
 1: {'apriori': 0.5445544554455446,
  'verosim': {0: {0.0: 0.5545454545454546, 1.0: 0.44545454545454544},
   1: {0.0: 0.8363636363636363, 1.0: 0.16363636363636364},
   2: {0.0: 0.7727272727272727, 1.0: 0.22727272727272727},
   3: {0.0: 0.2, 1.0: 0.8},
   4: {0.0: 0.6454545454545455, 1.0: 0.35454545454545455}}}}

In [12]:
for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

Fold 0:
  Train: index=[101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154
 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172
 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190
 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208
 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226
 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244
 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262
 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280
 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298
 299 300 301 302]
  Test:  index=[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34